# AAA Northeast Customer Clustering

## Assumptions
- We only have the current address of the customers, so we assume that most customers live in the same city most of the time.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read data
df = pd.read_csv('../project-AAA-northeast-member-clustering/data/member_sample.csv')
df = df[~df['Member Key'].isnull()].set_index('Member Key')
df.head()

,Unnamed: 0,Individual Key,Household Key,Member Flag,City,State - Grouped,ZIP5,ZIP9,FSV CMSI Flag,FSV Credit Card Flag,...,SC Vehicle Manufacturer Name,SC Vehicle Model Name,SVC Facility Name,SVC Facility Type,Total Cost,Tow Destination Latitude,Tow Destination Longitude,Tow Destination Name,Was Duplicated,Was Towed To AAR Referral
Member Key,,,,,,,,,,,,,,,,,,,,,
1500014.0,1,52211550,4500791,Y,WEST WARWICK,RI,2893,28933850,N,Y,...,TOYOTA,CAMRY,ASTRO WRECKER SERVICE,independent repair,32.5,41.0,-71.0,Aar / Johnson's Auto Service,0.0,1.0
1500014.0,2,52211550,4500791,Y,WEST WARWICK,RI,2893,28933850,N,Y,...,TOYOTA,CAMRY,Astro Wrecker Service,independent repair,30.0,0.0,0.0,NaN,0.0,0.0
1500014.0,3,52211550,4500791,Y,WEST WARWICK,RI,2893,28933850,N,Y,...,TOYOTA,CAMRY,ASTRO WRECKER SERVICE,independent repair,32.5,0.0,0.0,NaN,0.0,0.0
1500014.0,4,52211550,4500791,Y,WEST WARWICK,RI,2893,28933850,N,Y,...,TOYOTA,CAMRY,ASTRO WRECKER SERVICE,independent repair,30.0,0.0,0.0,NaN,0.0,0.0
1500014.0,5,52211550,4500791,Y,WEST WARWICK,RI,2893,28933850,N,Y,...,TOYOTA,CAMRY,AAA SNE RI LIGHT SERVICE,mobile battery service,53.0,0.0,0.0,NaN,0.0,0.0


## Data Preprocessing

### Column Selection
#### Identify Target Variables
- Products: 
    1. FSV CMSI
    2. FSV Credit Card
    3. FSV Deposit Program
    4. FSV Home Equity
    5. FSV ID Theft
    6. FSV Mortgage
    7. INS Client
    8. TRV Globalware
    9. New Mover
   
   
- Cost: Total Cost (Roadside data)

#### Select Key Columns
The rules of selecting columns
1. Use the granular column reasonable number of unique values when available (Ex: ZIP instead of state or ZIP9)
2. Use the more complete column within columns providing same information (Ex: gender instead of right_gender)
3. Use only one column within a group of correlated columns (Ex: Use tenure and drop Join AAA Date)information at household level.
4. Drop columns that satisfy the following criteria:
    - Have the same value on all or the majority of data (ex: Member Flag, Language)
    - Contain high missing values (ex: Reason Joined)
    - Should not be used for decision making (ex: Race)
    - Only have partial information (ex: Months from Join to Cancel is only for cancelled members)
    - Not informative (ex: Address Change Date)
    - Can't be aggregated to household level after data exploration (Ex: Education, Occupation, Vehicle)
    - Others (ex: Individual keys)
5. Keep only total cost from roadside data as the rest columns could not be aggregated

In [3]:
product_list = ['FSV CMSI Flag', 'FSV Credit Card Flag', 'FSV Deposit Program Flag', 
                'FSV Home Equity Flag', 'FSV ID Theft Flag', 'FSV Mortgage Flag',
                'INS Client Flag', 'TRV Globalware Flag', 'New Mover Flag']

cost = ['Total Cost']

selected_cols = ['Member Key', 'Household Key', 'ZIP', 'Number of Children', 
                 'Length Of Residence', 'Mail Responder', 'Home Owner', 'Income',  
                 'Dwelling Type', 'Credit Ranges', 'Do Not Direct Mail Solicit', 
                 'Email Available', 'ERS ENT Count Year 1', 'ERS ENT Count Year 2', 
                 'ERS ENT Count Year 3', 'ERS Member Cost Year 1', 'ERS Member Cost Year 2', 
                 'ERS Member Cost Year 3', 'Member Status', 'Member Tenure Years', 'Member Type', 
                 'Mosaic Household', 'Mosaic Global Household', 'kcl_B_IND_MosaicsGrouping']

### Individual Level
#### Aggregate Cost
Member data is duplicated for the same individual with multiple records of roadside data. To avoid duplication, aggregate roadside data (cost) to individual level by Member Key, the unique key for member data.

In [4]:
df_cost = df[cost].sum(level=0).fillna(0)
df_cost.head()

,Total Cost
Member Key,
115.0,29.0
116.0,0.0
241.0,32.5
242.0,0.0
243.0,28.0


In [5]:
df_cost.shape

(12341, 1)

#### Remove Duplication

In [6]:
df_member = df.loc[:, product_list + selected_cols
                  ].drop_duplicates().join(df_cost)
df_member.shape

(12229, 34)

In [7]:
df_member.head()

,FSV CMSI Flag,FSV Credit Card Flag,FSV Deposit Program Flag,FSV Home Equity Flag,FSV ID Theft Flag,FSV Mortgage Flag,INS Client Flag,TRV Globalware Flag,New Mover Flag,Member Key,...,ERS Member Cost Year 1,ERS Member Cost Year 2,ERS Member Cost Year 3,Member Status,Member Tenure Years,Member Type,Mosaic Household,Mosaic Global Household,kcl_B_IND_MosaicsGrouping,Total Cost
Member Key,,,,,,,,,,,,,,,,,,,,,
1500014.0,N,Y,N,N,N,N,N,N,N,NaN,...,0.00,0.0,65.0,ACTIVE,49.0,Primary,Senior Discounts,Low Income Elders,Golden Year Guardians,304.50
1500016.0,N,N,N,N,N,N,N,N,N,NaN,...,NaN,NaN,NaN,CANCELLED,13.0,Primary,Aging in Place,Comfortable Retirement,Autumn Years,0.00
1500019.0,N,N,N,N,N,N,N,N,N,NaN,...,58.85,117.7,0.0,ACTIVE,4.0,Primary,No Place Like Home,Routine Service Workers,Thriving Boomers,164.85
3221131.0,N,N,N,N,N,N,N,N,N,NaN,...,NaN,NaN,NaN,CANCELLED,59.0,Associate,No Place Like Home,Routine Service Workers,Thriving Boomers,0.00
4820433.0,N,N,N,N,N,N,N,N,N,NaN,...,NaN,NaN,NaN,CANCELLED,12.0,Associate,No Place Like Home,Routine Service Workers,Thriving Boomers,0.00


#### Remove Cancelled members

In [8]:
df_member['Member Status'].value_counts()

CANCELLED    6266
ACTIVE       5403
PENDING       560
Name: Member Status, dtype: int64

In [9]:
df_member = df_member[df_member['Member Status'] != 'CANCELLED']

#### Transfer Data Type
- Transfer to 1 and 0: Products, Mail Responder
- Transfer to numeric than use mean: Income, Credit Ranges, Number of Children
- Create new columns: Member Type

In [10]:
df_bool = df_member[product_list + 
                    ['Mail Responder']
                   ].replace(['Y', 'Yes'], 1
                            ).replace(['N', 'No'], 0)
df_bool.head()

,FSV CMSI Flag,FSV Credit Card Flag,FSV Deposit Program Flag,FSV Home Equity Flag,FSV ID Theft Flag,FSV Mortgage Flag,INS Client Flag,TRV Globalware Flag,New Mover Flag,Mail Responder
Member Key,,,,,,,,,,
1500014.0,0,1,0,0,0,0,0,0,0,1.0
1500019.0,0,0,0,0,0,0,0,0,0,1.0
15351060.0,0,0,0,0,0,0,0,0,0,1.0
1500025.0,0,0,0,0,0,0,1,0,0,1.0
1500028.0,0,0,0,0,0,0,0,1,0,1.0


In [11]:
df_children = df_member[['Number of Children']].replace({'No children': 0,
                                                         'One Child': 1,
                                                         'Two Children': 2,
                                                         'Three Children': 3,
                                                         'Four Children': 4,
                                                         'Five Children': 5,
                                                         'Six Children': 6})
df_children.head()

,Number of Children
Member Key,
1500014.0,1.0
1500019.0,0.0
15351060.0,0.0
1500025.0,0.0
1500028.0,1.0


In [12]:
df_income = df_member[['Income']].replace({'Under 10K': 5000,
                                           '10-19,999': 15000,
                                           '20-29,999': 25000,
                                           '30-39,999': 35000,
                                           '40-49,999': 45000, 
                                           '50-59,999': 55000,
                                           '60-69,999': 65000,
                                           '70-79,999': 75000,
                                           '80-89,999': 85000,
                                           '90-99,999': 95000,
                                           '100-149,999': 125000,
                                           '150 - 174,999': 162500, 
                                           '175 - 199,999': 187500,
                                           '200 - 249,999': 225000,
                                           '250K+': 250000})
df_income.head()

,Income
Member Key,
1500014.0,35000.0
1500019.0,55000.0
15351060.0,55000.0
1500025.0,55000.0
1500028.0,125000.0


In [13]:
df_credit = df_member[['Credit Ranges']].replace({'499 & Less': 250,
                                                  '500-549': 525,
                                                  '550-599': 575,
                                                  '600-649': 625,
                                                  '650-699': 675,
                                                  '700-749': 725,
                                                  '750-799': 775,
                                                  '800+': 800,
                                                  'Unknown': np.nan
                                                 })
df_credit.head()

,Credit Ranges
Member Key,
1500014.0,625.0
1500019.0,775.0
15351060.0,775.0
1500025.0,675.0
1500028.0,675.0


In [14]:
# Occup_unknown = df_member['Occupation Code'] == 'Unknown'
# df_occup = pd.DataFrame({'Occupation': pd.concat([df_member['Occupation Code'][~Occup_unknown],
#                                                   df_member['Occupation Group'][Occup_unknown]])})
# df_occup.head()

In [15]:
df_member[product_list + ['Mail Responder', 'Number of Children',
                          'Income', 'Credit Ranges']].tail()

,FSV CMSI Flag,FSV Credit Card Flag,FSV Deposit Program Flag,FSV Home Equity Flag,FSV ID Theft Flag,FSV Mortgage Flag,INS Client Flag,TRV Globalware Flag,New Mover Flag,Mail Responder,Number of Children,Income,Credit Ranges
Member Key,,,,,,,,,,,,,
1532316.0,N,N,N,N,N,N,N,N,N,Yes,Three Children,"20-29,999",600-649
1532325.0,Y,N,N,N,N,N,N,N,N,Yes,Two Children,"100-149,999",800+
13624040.0,N,Y,N,N,N,N,N,N,N,Yes,Two Children,"100-149,999",800+
1532344.0,Y,N,N,N,N,N,Y,N,Y,Yes,Two Children,"100-149,999",600-649
1532360.0,N,N,N,N,N,N,N,N,N,Yes,Two Children,"90-99,999",650-699


In [16]:
df_transform = df_member.drop(product_list + 
                              ['Mail Responder', 'Number of Children', 
                               'Income', 'Credit Ranges'],
                              axis = 1
                             ).join(df_bool).join(df_children).join(df_income).join(df_credit)
df_transform.tail()

,Member Key,Household Key,ZIP,Length Of Residence,Home Owner,Dwelling Type,Do Not Direct Mail Solicit,Email Available,ERS ENT Count Year 1,ERS ENT Count Year 2,...,FSV Home Equity Flag,FSV ID Theft Flag,FSV Mortgage Flag,INS Client Flag,TRV Globalware Flag,New Mover Flag,Mail Responder,Number of Children,Income,Credit Ranges
Member Key,,,,,,,,,,,,,,,,,,,,,
1532316.0,NaN,45466286,2809.0,12.0,Home Owner,SFDU,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,1.0,3.0,25000.0,625.0
1532325.0,NaN,13735475,2809.0,10.0,Home Owner,SFDU,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,1.0,2.0,125000.0,800.0
13624040.0,NaN,13735475,2809.0,10.0,Home Owner,SFDU,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1.0,2.0,125000.0,800.0
1532344.0,NaN,5462399,2879.0,0.0,NaN,SFDU,0.0,1.0,0.0,0.0,...,0,0,0,1,0,1,1.0,2.0,125000.0,625.0
1532360.0,NaN,8325571,2886.0,13.0,Home Owner,SFDU,0.0,1.0,2.0,0.0,...,0,0,0,0,0,0,1.0,2.0,95000.0,675.0


In [17]:
df_transform = df_transform.assign(PrimaryMember = 
                                       [1 if x == 'Primary' else 0 
                                            for x in df_transform['Member Type']],
                                   AssociateMember = 
                                       [1 if x == 'Associate' else 0 
                                            for x in df_transform['Member Type']])
df_transform[['Member Type', 'PrimaryMember', 'AssociateMember']].tail()

,Member Type,PrimaryMember,AssociateMember
Member Key,,,
1532316.0,Primary,1,0
1532325.0,Primary,1,0
13624040.0,Associate,0,1
1532344.0,Primary,1,0
1532360.0,Primary,1,0


In [18]:
df_transform = df_transform.drop(['Member Status', 'Member Type'], axis = 1)

In [19]:
df_transform.columns

Index(['Member Key', 'Household Key', 'ZIP', 'Length Of Residence',
       'Home Owner', 'Dwelling Type', 'Do Not Direct Mail Solicit',
       'Email Available', 'ERS ENT Count Year 1', 'ERS ENT Count Year 2',
       'ERS ENT Count Year 3', 'ERS Member Cost Year 1',
       'ERS Member Cost Year 2', 'ERS Member Cost Year 3',
       'Member Tenure Years', 'Mosaic Household', 'Mosaic Global Household',
       'kcl_B_IND_MosaicsGrouping', 'Total Cost', 'FSV CMSI Flag',
       'FSV Credit Card Flag', 'FSV Deposit Program Flag',
       'FSV Home Equity Flag', 'FSV ID Theft Flag', 'FSV Mortgage Flag',
       'INS Client Flag', 'TRV Globalware Flag', 'New Mover Flag',
       'Mail Responder', 'Number of Children', 'Income', 'Credit Ranges',
       'AssociateMember', 'PrimaryMember'],
      dtype='object')

### Household Level
#### Non-Numeric Columns
To aggregate non-numeric columns, we need to check if the non-numeric values within a household are too diverse. If the variables are consistent within the majority of household, we can use mode to get the variable that is representative to the household without misinterpreting the true information.

In [20]:
# Number of households with more than one values in each non-numeric columns
pd.DataFrame(df_transform.loc[:, [df_transform[x].dtype == 'O' for x in df_transform]
                             ].join(df_transform['Household Key']
                                   ).groupby(['Household Key']).nunique() > 1).sum()

Home Owner                   23
Dwelling Type                 0
Mosaic Household              4
Mosaic Global Household       1
kcl_B_IND_MosaicsGrouping     4
Household Key                 0
dtype: int64

From the result we can confidently aggregate Home Owner, Dwelling Type and Mosaic columns by taking mode, because only a handful of households have more than two values in these columns.

In [21]:
categorical_cols = ['ZIP', 'Home Owner', 'Dwelling Type', 'Mosaic Household', 
                    'Mosaic Global Household', 'kcl_B_IND_MosaicsGrouping']
df_household_cat = df_transform.fillna('Not Set'
                                       ).groupby(['Household Key']
                                                )[categorical_cols].agg(lambda x: 
                                                                        x.value_counts().index[0])
df_household_cat['ZIP'] = df_household_cat['ZIP'].astype('category')
df_household_cat.head()

,ZIP,Home Owner,Dwelling Type,Mosaic Household,Mosaic Global Household,kcl_B_IND_MosaicsGrouping
Household Key,,,,,,
875,2921.0,Not Set,SFDU,Couples with Clout,Bourgeois Prosperity,Power Elite
969,2919.0,Not Set,SFDU,Aging in Place,Comfortable Retirement,Autumn Years
8718,2910.0,Home Owner,SFDU,Rooted Flower Power,Post Industrial Survivors,Blue Sky Boomers
11524,2919.0,Home Owner,SFDU,No Place Like Home,Routine Service Workers,Thriving Boomers
13422,2816.0,Home Owner,SFDU,Unspoiled Splendor,Bourgeois Prosperity,Thriving Boomers


#### Numeric Columns

In [22]:
df_household_num = df_transform.groupby(['Household Key']
                                   ).agg({'Length Of Residence': 'mean',
                                          'Do Not Direct Mail Solicit': 'mean',
                                          'Email Available': 'mean', 
                                          'ERS ENT Count Year 1': ['sum', 'mean'],
                                          'ERS ENT Count Year 2': ['sum', 'mean'],
                                          'ERS ENT Count Year 3': ['sum', 'mean'],
                                          'ERS Member Cost Year 1': ['sum', 'mean'],
                                          'ERS Member Cost Year 2': ['sum', 'mean'],
                                          'ERS Member Cost Year 3': ['sum', 'mean'],
                                          'Member Tenure Years': 'mean',
                                          'Total Cost': 'sum',
                                          'Mail Responder': 'mean',
                                          'Number of Children': 'mean',
                                          'Income': 'mean',
                                          'Credit Ranges': 'mean',
                                          'AssociateMember': 'sum',
                                          'PrimaryMember': 'sum'
                                         })
df_household_num.columns = [' '.join(col).strip() for col in df_household_num.columns.values]
df_household_num.head()

,Credit Ranges mean,AssociateMember sum,ERS Member Cost Year 3 sum,ERS Member Cost Year 3 mean,ERS Member Cost Year 1 sum,ERS Member Cost Year 1 mean,Income mean,Email Available mean,Length Of Residence mean,PrimaryMember sum,...,ERS Member Cost Year 2 mean,Member Tenure Years mean,Mail Responder mean,ERS ENT Count Year 2 sum,ERS ENT Count Year 2 mean,Total Cost sum,Do Not Direct Mail Solicit mean,ERS ENT Count Year 1 sum,ERS ENT Count Year 1 mean,Number of Children mean
Household Key,,,,,,,,,,,,,,,,,,,,,
875,NaN,3,347.16,86.7900,117.70,29.425000,NaN,0.500000,NaN,1,...,14.712500,32.500000,NaN,1.0,0.250000,1063.20,0.0,2.0,0.500000,NaN
969,800.000000,3,301.19,75.2975,0.00,0.000000,95000.000000,0.750000,14.000000,1,...,39.212500,35.500000,1.0,2.0,0.500000,226.10,0.0,0.0,0.000000,0.000000
8718,800.000000,0,0.00,0.0000,0.00,0.000000,55000.000000,1.000000,15.000000,1,...,0.000000,13.000000,1.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.000000
11524,741.666667,2,159.00,53.0000,58.85,19.616667,138333.333333,1.000000,13.333333,1,...,92.633333,38.000000,1.0,5.0,1.666667,294.25,0.0,1.0,0.333333,0.666667
13422,675.000000,2,0.00,0.0000,58.85,19.616667,95000.000000,0.333333,15.000000,1,...,0.000000,27.666667,1.0,0.0,0.000000,118.85,0.0,1.0,0.333333,0.000000


#### Products

In [23]:
df_household_product = df_transform.groupby(['Household Key']
                                           )[product_list].agg(max)
df_household_product.head()

,FSV CMSI Flag,FSV Credit Card Flag,FSV Deposit Program Flag,FSV Home Equity Flag,FSV ID Theft Flag,FSV Mortgage Flag,INS Client Flag,TRV Globalware Flag,New Mover Flag
Household Key,,,,,,,,,
875,0,1,0,0,0,0,0,0,1
969,0,0,0,0,0,0,1,0,0
8718,0,0,0,0,0,0,0,1,0
11524,0,0,0,0,0,0,0,1,0
13422,0,0,0,0,0,0,1,1,0


In [24]:
df_household = df_household_cat.join(df_household_num).join(df_household_product)
df_household.head()

,ZIP,Home Owner,Dwelling Type,Mosaic Household,Mosaic Global Household,kcl_B_IND_MosaicsGrouping,Credit Ranges mean,AssociateMember sum,ERS Member Cost Year 3 sum,ERS Member Cost Year 3 mean,...,Number of Children mean,FSV CMSI Flag,FSV Credit Card Flag,FSV Deposit Program Flag,FSV Home Equity Flag,FSV ID Theft Flag,FSV Mortgage Flag,INS Client Flag,TRV Globalware Flag,New Mover Flag
Household Key,,,,,,,,,,,,,,,,,,,,,
875,2921.0,Not Set,SFDU,Couples with Clout,Bourgeois Prosperity,Power Elite,NaN,3,347.16,86.7900,...,NaN,0,1,0,0,0,0,0,0,1
969,2919.0,Not Set,SFDU,Aging in Place,Comfortable Retirement,Autumn Years,800.000000,3,301.19,75.2975,...,0.000000,0,0,0,0,0,0,1,0,0
8718,2910.0,Home Owner,SFDU,Rooted Flower Power,Post Industrial Survivors,Blue Sky Boomers,800.000000,0,0.00,0.0000,...,0.000000,0,0,0,0,0,0,0,1,0
11524,2919.0,Home Owner,SFDU,No Place Like Home,Routine Service Workers,Thriving Boomers,741.666667,2,159.00,53.0000,...,0.666667,0,0,0,0,0,0,0,1,0
13422,2816.0,Home Owner,SFDU,Unspoiled Splendor,Bourgeois Prosperity,Thriving Boomers,675.000000,2,0.00,0.0000,...,0.000000,0,0,0,0,0,0,1,1,0


In [25]:
df_household['Dwelling Type'].value_counts()

SFDU                                        2228
Not Set                                      849
Small or large multi-family w/apt number     366
P.O. BOX                                      61
Large multi-family w/o apt number              7
Name: Dwelling Type, dtype: int64

#### Imputation of Missing Values
The missing values in categorical columns have been filled with 'Not Set'. Now the missing values in numerical columns will be filled with median.

In [26]:
df_household.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3511 entries, 875 to 100079136
Data columns (total 38 columns):
ZIP                                3511 non-null category
Home Owner                         3511 non-null object
Dwelling Type                      3511 non-null object
Mosaic Household                   3511 non-null object
Mosaic Global Household            3511 non-null object
kcl_B_IND_MosaicsGrouping          3511 non-null object
Credit Ranges mean                 2844 non-null float64
AssociateMember sum                3511 non-null int64
ERS Member Cost Year 3 sum         3511 non-null float64
ERS Member Cost Year 3 mean        3511 non-null float64
ERS Member Cost Year 1 sum         3511 non-null float64
ERS Member Cost Year 1 mean        3511 non-null float64
Income mean                        2918 non-null float64
Email Available mean               3511 non-null float64
Length Of Residence mean           2924 non-null float64
PrimaryMember sum                  35

In [27]:
# Observation before filling NA
# sns.pairplot(df_household[['Income mean', 'Mail Responder mean', 'Length Of Residence mean',
#                            'Credit Ranges mean', 'Number of Children mean', 'Total Cost sum']].fillna(0))
# plt.show()

In [28]:
from sklearn.impute import SimpleImputer

In [29]:
numeric_features = df_household.select_dtypes(include=['float64', 'int64']).columns
imp_med = SimpleImputer(missing_values=np.nan, strategy='median')
df_num = pd.DataFrame(imp_med.fit_transform(df_household[numeric_features]))
df_num.columns = numeric_features
df_num.index = df_household.index
df_num.head()

,Credit Ranges mean,AssociateMember sum,ERS Member Cost Year 3 sum,ERS Member Cost Year 3 mean,ERS Member Cost Year 1 sum,ERS Member Cost Year 1 mean,Income mean,Email Available mean,Length Of Residence mean,PrimaryMember sum,...,Number of Children mean,FSV CMSI Flag,FSV Credit Card Flag,FSV Deposit Program Flag,FSV Home Equity Flag,FSV ID Theft Flag,FSV Mortgage Flag,INS Client Flag,TRV Globalware Flag,New Mover Flag
Household Key,,,,,,,,,,,,,,,,,,,,,
875,775.000000,3.0,347.16,86.7900,117.70,29.425000,95000.000000,0.500000,14.000000,1.0,...,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
969,800.000000,3.0,301.19,75.2975,0.00,0.000000,95000.000000,0.750000,14.000000,1.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8718,800.000000,0.0,0.00,0.0000,0.00,0.000000,55000.000000,1.000000,15.000000,1.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
11524,741.666667,2.0,159.00,53.0000,58.85,19.616667,138333.333333,1.000000,13.333333,1.0,...,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
13422,675.000000,2.0,0.00,0.0000,58.85,19.616667,95000.000000,0.333333,15.000000,1.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


#### One Hot Encoding

In [30]:
from sklearn.preprocessing import OneHotEncoder

In [31]:
categorical_features = df_household.select_dtypes(include=['category', 'object']).columns
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df_household[categorical_features])
df_cat = pd.DataFrame(enc.transform(df_household[categorical_features]).toarray())
df_cat.columns = enc.get_feature_names()
df_cat.index = df_household.index
df_cat.head()

,x0_1746.0,x0_1867.0,x0_1905.0,x0_2769.0,x0_2802.0,x0_2804.0,x0_2806.0,x0_2807.0,x0_2809.0,x0_2812.0,...,x5_Middle Class Melting Pot,x5_Not Set,x5_Pastoral Pride,x5_Power Elite,x5_Promising Families,x5_Significant Singles,x5_Singles and Starters,x5_Surburban Style,x5_Thriving Boomers,x5_Young City Solos
Household Key,,,,,,,,,,,,,,,,,,,,,
875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
13422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Final Dataset

In [32]:
X = df_num.join(df_cat)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3511 entries, 875 to 100079136
Columns: 229 entries, Credit Ranges mean to x5_Young City Solos
dtypes: float64(229)
memory usage: 6.3 MB


In [33]:
X.head()

,Credit Ranges mean,AssociateMember sum,ERS Member Cost Year 3 sum,ERS Member Cost Year 3 mean,ERS Member Cost Year 1 sum,ERS Member Cost Year 1 mean,Income mean,Email Available mean,Length Of Residence mean,PrimaryMember sum,...,x5_Middle Class Melting Pot,x5_Not Set,x5_Pastoral Pride,x5_Power Elite,x5_Promising Families,x5_Significant Singles,x5_Singles and Starters,x5_Surburban Style,x5_Thriving Boomers,x5_Young City Solos
Household Key,,,,,,,,,,,,,,,,,,,,,
875,775.000000,3.0,347.16,86.7900,117.70,29.425000,95000.000000,0.500000,14.000000,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
969,800.000000,3.0,301.19,75.2975,0.00,0.000000,95000.000000,0.750000,14.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8718,800.000000,0.0,0.00,0.0000,0.00,0.000000,55000.000000,1.000000,15.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11524,741.666667,2.0,159.00,53.0000,58.85,19.616667,138333.333333,1.000000,13.333333,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
13422,675.000000,2.0,0.00,0.0000,58.85,19.616667,95000.000000,0.333333,15.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [34]:
# Save data
X.to_csv('../project-AAA-northeast-member-clustering/data/processed_data.csv')